# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Elemento de lista
*   Elemento de lista



In [33]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

import re
import string


In [34]:
from google.colab import drive
drive.mount('/content/drive')
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/PLN/semana3"
os.chdir(DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [51]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


import pandas as pd

amazon_file = 'amazon5.txt'
imdb_file = 'imdb5.txt'
yelp_file = 'yelp5.txt'

def split_review_label(row):
    parts = re.split(r'\t+', row, maxsplit=1)
    return parts if len(parts) == 2 else [row, None]

with open(amazon_file, 'r', encoding='utf-8') as f:
    data = [split_review_label(line.strip()) for line in f]
df_amazon = pd.DataFrame(data, columns=['review', 'label'])
df_amazon.head(10)

with open(yelp_file, 'r', encoding='utf-8') as f:
    data = [split_review_label(line.strip()) for line in f]
df_yelp = pd.DataFrame(data, columns=['review', 'label'])
df_yelp.head(10)


def split_review_label_line(line):
    parts = line.rsplit(' ', 1)
    return parts if len(parts) == 2 else [line, None]

with open(imdb_file, 'r', encoding='utf-8') as f:
    data = [split_review_label_line(line.strip()) for line in f]
df_imdb = pd.DataFrame(data, columns=['review', 'label'])
df_imdb.head(10)

print(f"Amazon DataFrame tiene {len(df_amazon)} registros.")
print(f"IMDb DataFrame tiene {len(df_imdb)} registros.")
print(f"Yelp DataFrame tiene {len(df_yelp)} registros.")

df = pd.concat([df_amazon, df_imdb, df_yelp], ignore_index=True)

df['label'] = df['label'].astype(pd.Int64Dtype())
# *********** Aquí termina la sección de agregar código *************


Amazon DataFrame tiene 1000 registros.
IMDb DataFrame tiene 1000 registros.
Yelp DataFrame tiene 1000 registros.


In [52]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   Int64 
dtypes: Int64(1), object(1)
memory usage: 49.9+ KB


In [37]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


##**Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











In [38]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y dígitos
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenizar el texto
    words = text.split()
    # Eliminar stop words y aplicar lematización
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Unir las palabras procesadas de nuevo en una cadena
    return ' '.join(words)

# Aplicar el preprocesamiento a la columna 'review'
Xclean = df['review']
Xclean = Xclean.apply(preprocess_text)
# *********** Aquí termina la sección de agregar código *************

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [39]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


way plug u unless go converter
good case excellent value
great jawbone
tied charger conversation lasting minutesmajor problem
mic great


#**Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [42]:

# ************* Inicia la sección de agregar código:*****************************
X = Xclean
y = df['label']

# Partición de los datos en entrenamiento, validación y prueba
x_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)


# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X, y Train:', len(X_train), len(y_train))
print('X, y Val:', len(X_val), len(y_val))
print('X, y Test', len(X_test), len(y_test))

X, y Train: 2100 2100
X, y Val: 450 450
X, y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [ ]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


None


# *********** Aquí termina la sección de agregar código *************

c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [ ]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 7:**



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

#**Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 9:**



Reporte del mejor modelo.


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**